# Lesson 3: Chatbot Example

In this lesson, you will familiarize yourself with the chatbot example you will work on during this course. The example includes the tool definitions and execution, as well as the chatbot code. Make sure to interact with the chatbot at the end of this notebook.

## Import Libraries

In [13]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic
from openai import OpenAI

## Tool Functions

In [2]:
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the `papers` directory. The tool does not download the papers.  

In [3]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

In [4]:
search_papers("Transformers")

Results are saved in: papers/transformers/papers_info.json


['gr-qc/0612006v1',
 '1310.1984v2',
 '1605.08683v1',
 '1403.2188v1',
 '2204.07780v1']

The second tool looks for information about a specific paper across all topic directories inside the `papers` directory.

In [5]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        print(item)
        item_path = os.path.join(PAPER_DIR, item)
        print(item_path)
        
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            print(file_path)
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

In [54]:
print(extract_info('2412.09925v2'))

attention_in_transformer
papers/attention_in_transformer
papers/attention_in_transformer/papers_info.json
{
  "title": "Simulating Hard Attention Using Soft Attention",
  "authors": [
    "Andy Yang",
    "Lena Strobl",
    "David Chiang",
    "Dana Angluin"
  ],
  "summary": "We study conditions under which transformers using soft attention can\nsimulate hard attention, that is, effectively focus all attention on a subset\nof positions. First, we examine several subclasses of languages recognized by\nhard-attention transformers, which can be defined in variants of linear\ntemporal logic. We demonstrate how soft-attention transformers can compute\nformulas of these logics using unbounded positional embeddings or temperature\nscaling. Second, we demonstrate how temperature scaling allows softmax\ntransformers to simulate general hard-attention transformers, using a\ntemperature that depends on the minimum gap between the maximum attention\nscores and other attention scores.",
  "pdf_url

## Tool Schema

Here are the schema of each tool which you will provide to the LLM.

In [7]:
# tools = [
#     {
#         "name": "search_papers",
#         "description": "Search for papers on arXiv based on a topic and store their information.",
#         "input_schema": {
#             "type": "object",
#             "properties": {
#                 "topic": {
#                     "type": "string",
#                     "description": "The topic to search for"
#                 }, 
#                 "max_results": {
#                     "type": "integer",
#                     "description": "Maximum number of results to retrieve",
#                     "default": 5
#                 }
#             },
#             "required": ["topic"]
#         }
#     },
#     {
#         "name": "extract_info",
#         "description": "Search for information about a specific paper across all topic directories.",
#         "input_schema": {
#             "type": "object",
#             "properties": {
#                 "paper_id": {
#                     "type": "string",
#                     "description": "The ID of the paper to look for"
#                 }
#             },
#             "required": ["paper_id"]
#         }
#     }
# ]

In [8]:
# For OPENAI

# Convert your existing tools to OpenAI format
tools_openai = [
    {
        "type": "function",  # This is required for OpenAI
        "function": {
            "name": "search_papers",
            "description": "Search for papers on arXiv based on a topic and store their information.",
            "parameters": {  # Note: "parameters" instead of "input_schema"
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic to search for"
                    }, 
                    "max_results": {
                        "type": "integer",
                        "description": "Maximum number of results to retrieve",
                        "default": 5
                    }
                },
                "required": ["topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_info",
            "description": "Search for information about a specific paper across all topic directories.",
            "parameters": {
                "type": "object",
                "properties": {
                    "paper_id": {
                        "type": "string",
                        "description": "The ID of the paper to look for"
                    }
                },
                "required": ["paper_id"]
            }
        }
    }
]

## Tool Mapping

This code handles tool mapping and execution.

In [9]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

The chatbot handles the user's queries one by one, but it does not persist memory across the queries.

In [11]:
load_dotenv() 

True

In [14]:
load_dotenv() 
# client = anthropic.Anthropic(api_key="")
client = OpenAI()

### Query Processing

In [15]:
#This Python function `process_query(query)` seems to be handling a conversation between a user and an assistant using a messaging system. Here's a breakdown of what it does:

# Main functionalities:
# - Processes user queries and interacts with Claude AI model
# - Handles both text responses and tool usage
# - Executes tools and feeds results back to the conversation
# - Maintains conversation flow until query is fully processed
# - Prints assistant responses and tool execution details
# - Operates in a stateless manner (no memory between queries)



# def process_query(query):
    
#     messages = [{'role': 'user', 'content': query}]
    
#     response = client.messages.create(max_tokens = 2024,
#                                   model = 'claude-3-7-sonnet-20250219', 
#                                   tools = tools,
#                                   messages = messages)
    
    
#     process_query = True
#     while process_query:
#         assistant_content = []

#         for content in response.content:
#             if content.type == 'text':
                
#                 print(content.text)
#                 assistant_content.append(content)
                
#                 if len(response.content) == 1:
#                     process_query = False
            
#             elif content.type == 'tool_use':
                
#                 assistant_content.append(content)
#                 messages.append({'role': 'assistant', 'content': assistant_content})
                
#                 tool_id = content.id
#                 tool_args = content.input
#                 tool_name = content.name
#                 print(f"Calling tool {tool_name} with args {tool_args}")
                
#                 result = execute_tool(tool_name, tool_args)
#                 messages.append({"role": "user", 
#                                   "content": [
#                                       {
#                                           "type": "tool_result",
#                                           "tool_use_id": tool_id,
#                                           "content": result
#                                       }
#                                   ]
#                                 })
#                 response = client.messages.create(max_tokens = 2024,
#                                   model = 'claude-3-7-sonnet-20250219', 
#                                   tools = tools,
#                                   messages = messages) 
                
#                 if len(response.content) == 1 and response.content[0].type == "text":
#                     print(response.content[0].text)
#                     process_query = False

In [16]:
response = client.chat.completions.create(
        max_tokens=2024,
        model='gpt-4o',  # or other models like 'gpt-4-turbo', 'gpt-3.5-turbo'
        tools=tools_openai,  # Assumes 'tools' is defined in the outer scope
        tool_choice='auto',  # Let the model decide whether to use tools
        messages= [{'role': 'user', 'content': "What is your name?"}]
    )

In [17]:
response

ChatCompletion(id='chatcmpl-ByoDplrZBeRXrLN8MEUfNFYVR9YjX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am ChatGPT, a language model created by OpenAI.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1753832433, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_07871e2ad8', usage=CompletionUsage(completion_tokens=14, prompt_tokens=121, total_tokens=135, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [42]:
query = "Extract information about 2412.09925v2"

In [43]:
# Initialize the conversation with the user query
messages = [{'role': 'user', 'content': query}]

# Make the initial API call to OpenAI
response = client.chat.completions.create(
    max_tokens=2024,
    model='gpt-4o',  # or other models like 'gpt-4-turbo', 'gpt-3.5-turbo'
    tools=tools_openai,  # Assumes 'tools' is defined in the outer scope
    tool_choice='auto',  # Let the model decide whether to use tools
    messages=messages
)

In [44]:
assistant_message = response.choices[0].message
# print(assistant_message.content)
print(assistant_message.tool_calls)

[ChatCompletionMessageToolCall(id='call_XLYvN166eEwYISOm8siH43C6', function=Function(arguments='{"paper_id":"2412.09925v2"}', name='extract_info'), type='function')]


In [46]:
print(assistant_message.content)

None


In [47]:
# Add the assistant's message to conversation history
messages.append({
    'role': 'assistant',
    'content': assistant_message.content,
    'tool_calls': assistant_message.tool_calls
})

In [48]:
messages

[{'role': 'user', 'content': 'Extract information about 2412.09925v2'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_XLYvN166eEwYISOm8siH43C6', function=Function(arguments='{"paper_id":"2412.09925v2"}', name='extract_info'), type='function')]}]

In [49]:
# Process each tool call
for tool_call in assistant_message.tool_calls:
    tool_id = tool_call.id
    tool_name = tool_call.function.name
    # OpenAI provides arguments as a JSON string, so we need to parse it
    tool_args = json.loads(tool_call.function.arguments)
    
    print(f"Calling tool {tool_name} with args {tool_args}")
    
    # Execute the tool (assumes execute_tool function is defined elsewhere)
    result = execute_tool(tool_name, tool_args)

Calling tool extract_info with args {'paper_id': '2412.09925v2'}
attention_in_transformer
papers/attention_in_transformer
papers/attention_in_transformer/papers_info.json


In [51]:
print(result)

{
  "title": "Simulating Hard Attention Using Soft Attention",
  "authors": [
    "Andy Yang",
    "Lena Strobl",
    "David Chiang",
    "Dana Angluin"
  ],
  "summary": "We study conditions under which transformers using soft attention can\nsimulate hard attention, that is, effectively focus all attention on a subset\nof positions. First, we examine several subclasses of languages recognized by\nhard-attention transformers, which can be defined in variants of linear\ntemporal logic. We demonstrate how soft-attention transformers can compute\nformulas of these logics using unbounded positional embeddings or temperature\nscaling. Second, we demonstrate how temperature scaling allows softmax\ntransformers to simulate general hard-attention transformers, using a\ntemperature that depends on the minimum gap between the maximum attention\nscores and other attention scores.",
  "pdf_url": "http://arxiv.org/pdf/2412.09925v2",
  "published": "2024-12-13"
}


In [55]:
messages.append({
                    "role": "tool",
                    "tool_call_id": tool_id,
                    "content": json.dumps(result) if not isinstance(result, str) else result
                })

In [57]:
messages

[{'role': 'user', 'content': 'Extract information about 2412.09925v2'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_XLYvN166eEwYISOm8siH43C6', function=Function(arguments='{"paper_id":"2412.09925v2"}', name='extract_info'), type='function')]},
 {'role': 'tool',
  'tool_call_id': 'call_XLYvN166eEwYISOm8siH43C6',
  'content': '{\n  "title": "Simulating Hard Attention Using Soft Attention",\n  "authors": [\n    "Andy Yang",\n    "Lena Strobl",\n    "David Chiang",\n    "Dana Angluin"\n  ],\n  "summary": "We study conditions under which transformers using soft attention can\\nsimulate hard attention, that is, effectively focus all attention on a subset\\nof positions. First, we examine several subclasses of languages recognized by\\nhard-attention transformers, which can be defined in variants of linear\\ntemporal logic. We demonstrate how soft-attention transformers can compute\\nformulas of these logics using unbounded positional embe

In [58]:
response = client.chat.completions.create(
                max_tokens=2024,
                model='gpt-4o',
                tools=tools_openai,
                tool_choice='auto',
                messages=messages
            )

In [60]:
print(response.choices[0].message.tool_calls)

None


In [62]:
print(response.choices[0].message.content)

The paper titled **"Simulating Hard Attention Using Soft Attention"** is authored by Andy Yang, Lena Strobl, David Chiang, and Dana Angluin. Here is a summary of the paper:

The study explores conditions under which transformers using soft attention can simulate hard attention, meaning they can effectively focus all attention on a subset of positions. The research particularly examines several subclasses of languages recognized by hard-attention transformers, which can be described using variants of linear temporal logic. The paper demonstrates methods for how soft-attention transformers can compute formulas of these logics by employing unbounded positional embeddings or temperature scaling. Furthermore, it shows how temperature scaling enables softmax transformers to simulate general hard-attention transformers, by using a temperature that is dependent on the minimum gap between the maximum attention scores and other attention scores.

- **Published Date:** December 13, 2024
- **PDF U

In [ ]:
# from dotenv import load_dotenv
# import anthropic

# load_dotenv()
# anthropic = anthropic.Anthropic()

# def process_query(query):
#     messages = [{'role':'user', 'content':query}]
#     response = anthropic.messages.create(max_tokens = 2024,
#                                   model = 'claude-3-7-sonnet-20250219', 
#                                   tools = tools,
#                                   messages = messages)
#     process_query = True
#     while process_query:
#         assistant_content = []
#         for content in response.content:
#             if content.type =='text':
#                 print(content.text)
#                 assistant_content.append(content)
#                 if(len(response.content)==1):
#                     process_query= False
#             elif content.type == 'tool_use':
#                 assistant_content.append(content)
#                 messages.append({'role':'assistant', 'content':assistant_content})
#                 tool_id = content.id
#                 tool_args = content.input
#                 tool_name = content.name

#                 print(f"Calling tool {tool_name} with args {tool_args}")
                
#                 # Call a tool
#                 result = execute_tool(tool_name, tool_args)
#                 messages.append({"role": "user", 
#                                   "content": [
#                                       {
#                                           "type": "tool_result",
#                                           "tool_use_id":tool_id,
#                                           "content": result
#                                       }
#                                   ]
#                                 })
#                 response = anthropic.messages.create(max_tokens = 2024,
#                                   model = 'claude-3-7-sonnet-20250219', 
#                                   tools = tools,
#                                   messages = messages) 
                
#                 if(len(response.content)==1 and response.content[0].type == "text"):
#                     print(response.content[0].text)
#                     process_query= False


# def chat_loop():
#     print("Type your queries or 'quit' to exit.")
#     while True:
#         try:
#             query = input("\nQuery: ").strip()
#             if query.lower() == 'quit':
#                 break
    
#             process_query(query)
#             print("\n")
#         except Exception as e:
#             print(f"\nError: {str(e)}")

In [22]:
import json
from openai import OpenAI  # Import the OpenAI client library

def process_query(query):
    """
    Process a user query using OpenAI's GPT model, handling both text responses and tool calls.
    This function maintains a conversation flow until the query is fully processed.
    """
    
    # Initialize the conversation with the user query
    messages = [{'role': 'user', 'content': query}]
    
    # Make the initial API call to OpenAI
    response = client.chat.completions.create(
        max_tokens=2024,
        model='gpt-4o',  # or other models like 'gpt-4-turbo', 'gpt-3.5-turbo'
        tools=tools_openai,  # Assumes 'tools' is defined in the outer scope
        tool_choice='auto',  # Let the model decide whether to use tools
        messages=messages
    )
    
    process_query = True
    while process_query:
        # Get the assistant's message from the response
        assistant_message = response.choices[0].message
        
        # Check if the assistant is using tools or providing a text response
        if assistant_message.tool_calls:
            # Assistant wants to use tools
            
            # If there's any text content alongside the tool calls, print it
            if assistant_message.content:
                print(assistant_message.content)
            
            # Add the assistant's message to conversation history
            messages.append({
                'role': 'assistant',
                'content': assistant_message.content,
                'tool_calls': assistant_message.tool_calls
            })
            
            # Process each tool call
            for tool_call in assistant_message.tool_calls:
                tool_id = tool_call.id
                tool_name = tool_call.function.name
                # OpenAI provides arguments as a JSON string, so we need to parse it
                tool_args = json.loads(tool_call.function.arguments)
                
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                # Execute the tool (assumes execute_tool function is defined elsewhere)
                result = execute_tool(tool_name, tool_args)
                
                # Add the tool result to the conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_id,
                    "content": json.dumps(result) if not isinstance(result, str) else result
                })
            
            # Make another API call with the updated conversation including tool results
            response = client.chat.completions.create(
                max_tokens=2024,
                model='gpt-4o',
                tools=tools_openai,
                tool_choice='auto',
                messages=messages
            )
            
            # Check if this response has only text content (no more tool calls)
            if not response.choices[0].message.tool_calls:
                print(response.choices[0].message.content)
                process_query = False
                
        else:
            # Assistant provided a text response (no tool calls)
            if assistant_message.content:
                print(assistant_message.content)
            process_query = False


### Chat Loop

In [23]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Feel free to interact with the chatbot. Here's an example query: 

- Search for 2 papers on "LLM interpretability"

To access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L3`.

In [24]:
# 2 papers on attention in transformer

In [26]:
chat_loop()

Type your queries or 'quit' to exit.
Narendra Modi is an Indian politician who has been serving as the Prime Minister of India since May 2014. He is a member of the Bharatiya Janata Party (BJP) and the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist volunteer organization. Here are some key points about Narendra Modi:

1. **Early Life and Education**: Narendra Damodardas Modi was born on September 17, 1950, in Vadnagar, Gujarat, India. He was raised in a modest family and helped his father sell tea as a child. Modi earned a Master’s degree in Political Science from Gujarat University.

2. **Political Career**:
   - **Chief Minister of Gujarat**: Modi served as the Chief Minister of the Indian state of Gujarat from 2001 to 2014. During his tenure, he was credited with fostering economic growth and infrastructure development but faced criticism for his handling of the 2002 Gujarat riots.
   - **Prime Minister of India**: Modi led the BJP to a landslide victory in the 2014 general 

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the <code>requirements.txt</code> file or the <code>papers</code> folder: </b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and finally 3) click on <em>"L3"</em>.
</div>

In the next lessons, you will take out the tool definitions to wrap them in an MCP server. Then you will create an MCP client inside the chatbot to make the chatbot MCP compatible.  

## Resources

[Guide on how to implement tool use](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>